In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from UDCUtils import UDCUtils

utils = UDCUtils()

In [3]:
from pinecone import Pinecone, ServerlessSpec

In [4]:
pinecone_api_key = utils.get_pinecone_api_key()
print(pinecone_api_key)

fcd3e082-248a-4a31-b1c0-80c255c9d687


In [5]:
from sentence_transformers import SentenceTransformer
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm

In [9]:
index_name = utils.create_dlai_index_name("idx-log1")
print(index_name)

idx-log1-118d1-7f10-4f3c-a4a6-41d7caaf84cc


In [10]:
pinecone = Pinecone(
    api_key=pinecone_api_key
)

In [11]:
pinecone.create_index(
    name=index_name,
    dimension=256,
    metric='cosine',
    spec=ServerlessSpec(cloud="aws", region="us-east-1")
)

INDEX = pinecone.Index(index_name)

In [13]:
#!wget -q --show-progress -O training.tar.zip "https://www.dropbox.com/scl/fi/rihfngx4ju5pzjzjj7u9z/lesson6.tar.zip?rlkey=rct9a9bo8euqgshrk8wiq2orh&dl=1"

#!tar -xzvf training.tar.zip

#!tar -xzvf lesson6.tar

x sample.log
x training.txt


In [12]:
!ls -lrt training.txt sample.log

-rw-r--r--@ 1 sana  staff   7342 Dec 11  2023 training.txt
-rw-r--r--  1 sana  staff  14272 Dec 11  2023 sample.log


In [13]:
!head -5 sample.log

Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0]
Apr 15 2014 09:34:34 EDT: %ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0]
Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100: access-list inside denied udp inside/172.29.2.101(1039) -> outside/192.0.2.10(53) hit-cnt 1 first hit [0xd820e56a, 0x0]
Apr 24 2013 16:00:27 INT-FW01 : %ASA-6-106100: access-list inside permitted udp inside/172.29.2.3(1065) -> outside/192.0.2.57(53) hit-cnt 144 300-second interval [0xe982c7a4, 0x0]


In [14]:
!head -5 training.txt

Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 1.0
Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 0.9
Apr 15 2014 09:34:34 EDT: %ASA-session-5-106100: access-list acl_in permitted tcp inside/10.1.2.16(2241) -> outside/192.0.0.89(2000) hit-cnt 1 first hit [0x71a87d94, 0x0] ^ Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0] ^ 0.8
Apr 24 2013 16:00:28 INT-FW01 : %ASA-6-106100: access-list inside denied udp inside/172.29.2.1

### Check cuda and setup the model

In [15]:
import torch
device = 'cuda' if torch.cuda.is_available() == True else 'cpu'
print (device)

cpu


In [52]:
#BERT model for word embeddings
from sentence_transformers import InputExample, util, losses, models
#1
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=768)
#2
pooling_model = models.Pooling(word_embedding_dimension=word_embedding_model.get_word_embedding_dimension())
#3
dense_model = models.Dense(
    in_features=pooling_model.get_sentence_embedding_dimension(),
    out_features=256,
    activation_function=nn.Tanh()
)
#model
#model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model], device=device)
model = SentenceTransformer(modules=[word_embedding_model, pooling_model, dense_model], device="mps") #change device to mps for training the model

In [53]:
train_examples = []

with open('training.txt', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            a,b, label = line.split('^')
            #print(a,b,label)
            train_examples.append(InputExample(texts=[a,b], label=float(label)))

In [54]:
len(train_examples)

26

In [55]:
#import pandas as pd
#df = pd.DataFrame(train_examples)
#df.head()

In [56]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)
train_loss_fn = losses.CosineSimilarityLoss(model)

In [20]:
#pip install accelerate -U

In [58]:
import pickle
from sentence_transformers import SentenceTransformerTrainingArguments, SentenceTransformerTrainer
from datasets import Dataset

load_pretrained_model = True

warmup_steps=100

train_dataset = Dataset.from_pandas(df)

if load_pretrained_model:
    trained_model_file = open('pickle/pretrained_model','rb')
    db = pickle.load(trained_model_file)
    trained_model_file.close()
else:
    model.fit(train_objectives=[(train_dataloader, train_loss_fn)], epochs=16, warmup_steps=warmup_steps)

In [59]:
help(model.fit)

Help on method fit in module sentence_transformers.fit_mixin:

fit(train_objectives: Iterable[Tuple[torch.utils.data.dataloader.DataLoader, torch.nn.modules.module.Module]], evaluator: sentence_transformers.evaluation.SentenceEvaluator.SentenceEvaluator = None, epochs: int = 1, steps_per_epoch=None, scheduler: str = 'WarmupLinear', warmup_steps: int = 10000, optimizer_class: Type[torch.optim.optimizer.Optimizer] = <class 'torch.optim.adamw.AdamW'>, optimizer_params: Dict[str, object] = {'lr': 2e-05}, weight_decay: float = 0.01, evaluation_steps: int = 0, output_path: str = None, save_best_model: bool = True, max_grad_norm: float = 1, use_amp: bool = False, callback: Callable[[float, int, int], NoneType] = None, show_progress_bar: bool = True, checkpoint_path: str = None, checkpoint_save_steps: int = 500, checkpoint_save_total_limit: int = 0) -> None method of sentence_transformers.SentenceTransformer.SentenceTransformer instance
    Deprecated training method from before Sentence Trans

In [60]:
samples = []

with open('sample.log', "r") as f:
    lines = f.readlines()
    for line in lines:
        line = line.strip()
        if line:
            samples.append(line)

In [61]:
len(samples)

90

### Create embeddings and upsert to pinecone

In [62]:
emb = model.encode(samples)

prepped = []

for i in tqdm(range(len(samples))):
    v = {'id':f'{i}', 'values':emb[i].tolist() ,'metadata':{'log':samples[i]}}
    prepped.append(v)

print(len(prepped))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
100%|████████████████████████████████████████| 90/90 [00:00<00:00, 73412.56it/s]

90


In [63]:
INDEX.upsert(prepped)

{'upserted_count': 90}

In [64]:
good_log_line = samples[0]
print(good_log_line)

Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]


In [65]:
from time import sleep

In [66]:
results = []

while len(results)==0:
    sleep(2)
    response = INDEX.query(
        vector=emb[0].tolist(),
        top_k=100,
        include_metadata=True
    )
    results = response["matches"]
    print(".:. ",end="")

.:. 

In [67]:
#print(results)

[{'id': '0',
 'metadata': {'log': 'Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src '
                     'dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by '
                     'access-group "acl_dmz" [0xe3aab522, 0x0]'},
 'score': 1.0,
 'values': []}, {'id': '1',
 'metadata': {'log': 'Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src '
                     'dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, '
                     'code 0, by access-group "acl_dmz" [0xe3aab522, 0x0]'},
 'score': 0.995257795,
 'values': []}, {'id': '34',
 'metadata': {'log': 'Apr 30 2013 09:23:40: %ASA-4-106023: Deny tcp src '
                     'outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by '
                     'access-group "acl_out" [0x71761f18, 0x0]'},
 'score': 0.984457195,
 'values': []}, {'id': '35',
 'metadata': {'log': 'Apr 30 2013 09:23:41: %ASA-4-106023: Deny tcp src '
                     'outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by '
                     'acce

In [68]:
for i in range(0,10) :
  print(f"{round(results[i]['score'], 4)}\t{results[i]['metadata']['log']}")

1.0	Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 by access-group "acl_dmz" [0xe3aab522, 0x0]
0.9953	Apr 15 2013 09:36:50: %ASA-4-106023: Deny tcp src dmz:10.1.2.30/63016 dst outside:192.0.0.8/53 type 3, code 0, by access-group "acl_dmz" [0xe3aab522, 0x0]
0.9845	Apr 30 2013 09:23:40: %ASA-4-106023: Deny tcp src outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by access-group "acl_out" [0x71761f18, 0x0]
0.9839	Apr 30 2013 09:23:41: %ASA-4-106023: Deny tcp src outside:192.0.2.126/53638 dst inside:10.0.0.132/8111 by access-group "acl_out" [0x71761f18, 0x0]
0.9831	Dec 11 2018 08:01:39 <IP>: %ASA-4-106023: Deny udp src dmz:192.168.1.34/5679 dst outside:192.0.0.12/5000 by access-group "dmz" [0x123a465e, 0x8c20f21]
0.9806	Dec 11 2018 08:01:24 <IP>: %ASA-4-106023: Deny udp src dmz:192.168.1.33/5555 dst outside:192.0.0.12/53 by access-group "dmz" [0x123a465e, 0x4c7bf613]
0.9806	Dec 11 2018 08:01:24 <IP>: %ASA-4-106023: Deny udp src dmz:192.16

In [69]:
last_line = len(results)-1

print(f"{round(results[last_line]['score'])}\t{results[last_line]['metadata']['log']}")

1	dec 31, 2021 09:18:59: %ASA-4-434005: seg fault detected in the matrix
